In [3]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import pyarrow as pa
import dask
import fsspec
from datetime import datetime, timedelta

import dask.dataframe as dd
from dask.distributed import Client
import os
import xarray as xr
import glob


In [4]:
dir_files = [os.path.join("short_range_parquet_2files", files) for files in os.listdir("short_range_parquet_2files")]
print(dir_files)

['short_range_parquet_2files/nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f001.conus.nc', 'short_range_parquet_2files/nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f002.conus.nc']


In [6]:
# Create list 
individual_files = []
# Loop through each file in the list
for i in dir_files:

    # Load a single dataset
    timestep_ds = xr.open_dataset(i)

    # Add the dataset to the list
    individual_files.append(timestep_ds)

# Combine individual datasets into a single xarray along the 'time' dimension
ds = xr.concat(individual_files, dim='time')

ds


<xarray.Dataset>
Dimensions:         (time: 2, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 2022-09-11T01:00:00 2022-09-11T02:0...
  * reference_time  (reference_time) datetime64[ns] 2022-09-11
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
Data variables:
    crs             (time) |S1 b'' b''
    streamflow      (time, feature_id) float64 0.18 0.01 0.01 ... 0.0 0.0 0.0
    nudge           (time, feature_id) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    velocity        (time, feature_id) float64 0.08 0.1 0.1 ... 0.02 0.0 0.02
    qSfcLatRunoff   (time, feature_id) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    qBucket         (time, feature_id) float64 0.00341 0.00858 ... 0.00016
    qBtmVertRunoff  (time, feature_id) float64 12.18 26.58 9.311 ... 0.531 0.758
Attributes: (12/19)
    TITLE:                      OUTPUT FROM NWM v2.2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2022-09-11_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2022-09-11_01:00:00
    ...                         ...
    model_configuration:        short_range
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

In [8]:
df = ds.to_dataframe()
d_ser = df['streamflow']
d_ser


time                 reference_time  feature_id
2022-09-11 01:00:00  2022-09-11      101           0.18
                                     179           0.01
                                     181           0.01
                                     183           0.01
                                     185           0.01
                                                   ... 
2022-09-11 02:00:00  2022-09-11      1180001800    0.00
                                     1180001801    0.00
                                     1180001802    0.00
                                     1180001803    0.00
                                     1180001804    0.00
Name: streamflow, Length: 5553476, dtype: float64

In [9]:
df = pd.Series.to_frame(d_ser)
df

streamflow
time                reference_time feature_id            
2022-09-11 01:00:00 2022-09-11     101               0.18
                                   179               0.01
                                   181               0.01
                                   183               0.01
                                   185               0.01
...                                                   ...
2022-09-11 02:00:00 2022-09-11     1180001800        0.00
                                   1180001801        0.00
                                   1180001802        0.00
                                   1180001803        0.00
                                   1180001804        0.00

[5553476 rows x 1 columns]

In [10]:
# To write the dataframe to single parquet file using pyarrow and snappy compression

df.to_parquet("data/multiple_files_short_range_18files_new.parquet"
             ,  engine="pyarrow"
             , compression="snappy"
             )



In [11]:
# Read parquet file 
ddf = dd.read_parquet(
    "data/multiple_files_short_range_18files_new.parquet"
    , storage_options={"anon": True}
    #, columns=["streamflow", "velocity"]
    #index="reference_time",

)

#result = ddf.shape[0].compute()
result = ddf.compute()
#%time result
%time ddf.loc['2022-09-11 02:00'].compute()
#pip install parquet-tools
#parquet-tools --help

CPU times: user 1.55 s, sys: 369 ms, total: 1.91 s
Wall time: 1.9 s


streamflow
time                reference_time feature_id            
2022-09-11 02:00:00 2022-09-11     101               0.18
                                   179               0.01
                                   181               0.01
                                   183               0.01
                                   185               0.01
...                                                   ...
                                   1180001800        0.00
                                   1180001801        0.00
                                   1180001802        0.00
                                   1180001803        0.00
                                   1180001804        0.00

[2776738 rows x 1 columns]